# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Цель исследования:** разработать модель, которая определяет рыночную стоимость автомобиля на основе его технических характеристик и комплектации. Модель будет использоваться в новом приложении сервиса по продаже автомобилей с пробегом «Не бит, не крашен». 

**Ход исследования:**

1. Загрузка и изучение данных.

2. Предобработка данных: все пропущенные значения будут заполнены или удалены, в зависимости от их характера и количества. Временные данные будут преобразованы в подходящий формат.

3. Подготовка выборок для обучения моделей: данные будут разделены на обучающую, валидационную и тестовую выборки.

4. Обучение моделей: будут обучены различные модели, включая модель градиентного бустинга LightGBM и как минимум одну модель, не основанную на бустинге. Для каждой модели будут подобраны оптимальные гиперпараметры.

5. Сравнение моделей: анализ обученных моделей по трем критериям: время обучения, время предсказания и качество предсказаний (RMSE).

6. Выбор лучшей модели: будет выбрана лучшая модель, которая затем будет проверена на тестовой выборке.

7. Заключение:  выводы об эффективности разработанной модели и рекомендации по ее дальнейшему использованию и возможному улучшению.

## Подготовка данных

### Импорт библиотек

In [1]:
pip install --upgrade category_encoders

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from category_encoders import MEstimateEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from time import time
from sklearn.preprocessing import LabelEncoder
from scipy import stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# import warnings
# warnings.filterwarnings("ignore")

### Загрузка данных

In [3]:
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')

### Изучение данных

In [4]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [6]:
# Проверка на наличие пропусков
print("Missing values:")
data.isnull().sum()

Missing values:


DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [7]:
round(data.isnull().mean()*100, 2)

DateCrawled           0.00
Price                 0.00
VehicleType          10.58
RegistrationYear      0.00
Gearbox               5.60
Power                 0.00
Model                 5.56
Kilometer             0.00
RegistrationMonth     0.00
FuelType              9.28
Brand                 0.00
Repaired             20.08
DateCreated           0.00
NumberOfPictures      0.00
PostalCode            0.00
LastSeen              0.00
dtype: float64

In [8]:
data.corr(numeric_only=True)

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
RegistrationYear,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
Kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
RegistrationMonth,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


In [9]:
# Проверка на дубликаты
data.duplicated().sum()

4

### Вывод

После первичного анализа данных, мы выявили, что датасет состоит из 354369 записей и 16 столбцов. Среди признаков имеются как числовые, так и категориальные, и некоторые из них имеют пропущенные значения. Признаки 'VehicleType', 'Gearbox', 'Model', 'FuelType' и 'Repaired' имеют значительное количество пропусков, которые нуждаются в дальнейшей обработке.

## Предобработка данных

### Удаление ненужных столбцов

In [10]:
data['DateCrawled'].unique()

array(['2016-03-24 11:52:17', '2016-03-24 10:58:45',
       '2016-03-14 12:52:21', ..., '2016-03-21 09:50:58',
       '2016-03-14 17:48:27', '2016-03-19 18:57:12'], dtype=object)

In [11]:
data_with_date = data.copy()
data = data.drop(["DateCrawled","DateCreated","LastSeen","NumberOfPictures","PostalCode"],axis = 1)

### Переименовывание столбцов

In [12]:
data.columns = data.columns.str.replace(r"([A-Z])", r" \1", regex=True).str.lower().str.replace(' ', '_', regex=True).str[1:]
data.head()

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


### Заполнение пропусков

In [13]:
# Заменим пропущенные значения новой категорией "unknown"
for column in ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired']:
    data[column].fillna('unknown', inplace=True)
data.isnull().sum()

price                 0
vehicle_type          0
registration_year     0
gearbox               0
power                 0
model                 0
kilometer             0
registration_month    0
fuel_type             0
brand                 0
repaired              0
dtype: int64

### Удаление дубликатов

In [14]:
data = data.drop_duplicates()
data.duplicated().sum()

0

### Замена дублирующихся значений в категориях

In [15]:
data['fuel_type'].unique()

array(['petrol', 'gasoline', 'unknown', 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [16]:
# gasoline то же самое, что и petrol, поэтому переименуем
data['fuel_type'] = data['fuel_type'].replace(['gasoline'],'petrol')

In [17]:
data.loc[data['registration_month'] == 0, 'registration_month'] = 1

## Обработка выбросов

In [18]:
data.describe()

,price,registration_year,power,kilometer,registration_month
count,326826.000000,326826.000000,326826.000000,326826.000000,326826.000000
mean,4403.747330,2004.230985,110.244705,128144.073605,5.802589
std,4518.209808,91.120018,195.886373,37947.663920,3.571068
min,0.000000,1000.000000,0.000000,5000.000000,1.000000
25%,1000.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6350.000000,2008.000000,141.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


### Год регистрации

In [19]:
# Можно предположить, что реальные годы регистрации находятся в интервале
# 1885 (изобретение автомобиля) - 2016 (дата скачивания анкеты из базы)
data = data[(data['registration_year'] >= 1885) & (data['registration_year'] <= 2016)]

### Мощность двигателя

In [20]:
# Предполагаем, что реалистичная мощность двигателя находится в диапазоне от 1 до 2000 л.с.
data = data[(data['power'] > 0) & (data['power'] <= 2000)]

### Аномалии в цене 

In [21]:
# Предполагаем, что автомобили с ценой 0 - это ошибка, поэтому точно удаляем эти записи.
# Скорее всего, цены ниже 100 евро тоже ошибочны, поэтому удалим их.
data = data[data['price'] > 100]

### Удаление дубликатов v2

In [22]:
data.duplicated().sum()

271

In [23]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [24]:
# Проверяем результат
data.describe()

,price,registration_year,power,kilometer,registration_month
count,271651.000000,271651.000000,271651.000000,271651.000000,271651.000000
mean,4865.601220,2002.799798,121.532297,128187.288101,6.044292
std,4615.461251,6.605783,63.820391,36849.275900,3.487775
min,101.000000,1910.000000,1.000000,5000.000000,1.000000
25%,1350.000000,1999.000000,77.000000,125000.000000,3.000000
50%,3200.000000,2003.000000,111.000000,150000.000000,6.000000
75%,6999.000000,2007.000000,150.000000,150000.000000,9.000000
max,20000.000000,2016.000000,2000.000000,150000.000000,12.000000


### Вывод

Мы выполнили предобработку данных, включающую обработку пропущенных значений и аномалий. Пропуски в категориальных признаках были выделены в отдельную колонку 'unknown'.

При обработке аномалий мы обнаружили необычные значения в признаках 'registration_year', 'power' и 'price'. Было принято решение ограничить годы регистрации интервалом 1885-2023, мощность автомобилей интервалом 1-2000 л.с., и исключить автомобили с нулевой ценой.

После этих шагов мы получили более чистый датасет, состоящий из 271 651 объектов, который можно использовать для построения моделей машинного обучения.

## Обучение моделей

### Разделение данных на обучающую, валидационную и тестовую выборки

In [25]:
trainX, tempX, trainY, tempY = train_test_split(data.drop("price", axis=1), 
                                                data["price"], 
                                                test_size=0.4, 
                                                random_state=12345)

valX, testX, valY, testY = train_test_split(tempX, tempY, 
                                            test_size=0.5, 
                                            random_state=12345)

In [26]:
categorical_cols_ohe = ["vehicle_type", "gearbox", "fuel_type", "repaired"]
target_encode_cols = ["brand", "model"]

# Создаем трансформеры
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
me = MEstimateEncoder(cols=target_encode_cols)

# Создаем пайплайн для целевого кодирования
target_encode_pipeline = make_pipeline(me)

# Создаем ColumnTransformer, который включает все преобразования
preprocessor = make_column_transformer(
    (ohe, categorical_cols_ohe),
    (target_encode_pipeline, target_encode_cols),
    remainder='passthrough'
)

# Обучаем ColumnTransformer на тренировочных данных и преобразуем тренировочные данные
trainX_encoded = preprocessor.fit_transform(trainX, trainY)

# Преобразуем валидационные и тестовые данные
valX_encoded = preprocessor.transform(valX)
testX_encoded = preprocessor.transform(testX)

# Получаем список названий признаков, учитывая, что для признаков с целевым кодированием
# названия признаков будут такими же, как в target_encode_cols
feature_names = (list(preprocessor.named_transformers_['onehotencoder'].get_feature_names_out(categorical_cols_ohe)) 
                 + target_encode_cols + [c for c in trainX.columns if c not in (categorical_cols_ohe 
                                                                                + target_encode_cols)])

# Конвертируем numpy массивы обратно в датафреймы
trainX_encoded = pd.DataFrame(trainX_encoded, columns = feature_names)
valX_encoded = pd.DataFrame(valX_encoded, columns = feature_names)
testX_encoded = pd.DataFrame(testX_encoded, columns = feature_names)

trainX_encoded.head()

,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_unknown,vehicle_type_wagon,gearbox_manual,gearbox_unknown,...,fuel_type_petrol,fuel_type_unknown,repaired_unknown,repaired_yes,brand,model,registration_year,power,kilometer,registration_month
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,6640.479568,5369.763408,1997.0,170.0,150000.0,10.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,3533.302057,2929.579824,2001.0,60.0,100000.0,9.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,3953.215082,3732.661544,2000.0,191.0,125000.0,7.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,3078.309752,1207.896228,2000.0,144.0,150000.0,3.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,4844.423247,1499.714866,2000.0,75.0,150000.0,1.0


In [27]:
print(trainX_encoded.shape)
print(valX_encoded.shape)
print(testX_encoded.shape)

(162990, 24)
(54330, 24)
(54331, 24)


In [28]:
data_lightgbm = data.astype({"vehicle_type":'category',
                                  "gearbox":'category',
                                  "model":'category',
                                  "fuel_type":'category',
                                  "brand":'category',
                                  "repaired":'category'})

In [29]:
(trainX_lightgbm, tempX_lightgbm,
 trainY_lightgbm, tempY_lightgbm) = train_test_split(data_lightgbm.drop("price",axis = 1),
                                                     data_lightgbm["price"],
                                                     test_size = 0.4,
                                                     random_state = 12345)

(testX_lightgbm, valX_lightgbm,
 testY_lightgbm, valY_lightgbm) = train_test_split(tempX_lightgbm, tempY_lightgbm,
                                                   test_size = 0.5,
                                                   random_state = 12345)


### Линенейная регрессия

In [30]:
# Создаем объект стандартизатора
scaler = StandardScaler()

# Обучаем стандартизатор на обучающем наборе и применяем его к обучающему и валидационному набору
trainX_scaled = scaler.fit_transform(trainX_encoded)
valX_scaled = scaler.transform(valX_encoded)
testX_scaled = scaler.transform(testX_encoded)

In [31]:
%%time

linear_model = LinearRegression()

# Проведение кросс-валидации с 5 фолдами
cross_val_scores = cross_val_score(linear_model, trainX_scaled, trainY, cv=5, scoring='neg_root_mean_squared_error')

cross_val_rmse = -cross_val_scores

print("Cross-validation RMSE for Linear Regression: ", cross_val_rmse)

# Вывод среднего значения и стандартного отклонения RMSE по всем фолдам
print("Mean cross-validation RMSE: ", cross_val_rmse.mean())
print("Standard Deviation of cross-validation RMSE: ", cross_val_rmse.std())


Cross-validation RMSE for Linear Regression:  [2850.63007681 2912.91538973 2887.22403066 2879.1774058  2894.27645333]
Mean cross-validation RMSE:  2884.844671267395
Standard Deviation of cross-validation RMSE:  20.422249977443098
CPU times: user 2.42 s, sys: 331 ms, total: 2.76 s
Wall time: 431 ms


In [32]:
%%time

# Создаем и обучаем модель
linear_model = LinearRegression()
linear_model.fit(trainX_scaled, trainY)

CPU times: user 671 ms, sys: 56.3 ms, total: 727 ms
Wall time: 107 ms


LinearRegression()

In [33]:
%%time

# Проверяем качество модели на валидационной выборке
predictions = linear_model.predict(valX_scaled)

# Расчитываем RMSE
rmse = mean_squared_error(valY, predictions, squared=False)

print("RMSE for Linear Regression: ", rmse)

RMSE for Linear Regression:  2912.013998155108
CPU times: user 34 ms, sys: 1.56 ms, total: 35.6 ms
Wall time: 7.98 ms


### LightGBM

In [34]:
%%time
# Параметры для поиска
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [-1, 10],
}

lgbm = LGBMRegressor(random_state=12345)

grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid,
                           cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Обучаем модель
grid_search.fit(trainX_lightgbm, trainY_lightgbm)

# лучшие гиперпараметры
best_parameters = grid_search.best_params_
print("Best parameters: ", best_parameters)

Best parameters:  {'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 100}
CPU times: user 2.85 s, sys: 1.31 s, total: 4.16 s
Wall time: 7.06 s


In [35]:
%%time
# Обучаем модель с использованием лучших гиперпараметров
model_lgbm_best = LGBMRegressor(n_estimators=best_parameters['n_estimators'],
                                learning_rate=best_parameters['learning_rate'],
                                max_depth=best_parameters['max_depth'],
                                loss_function='RMSE', 
                                random_seed=12345)

model_lgbm_best.fit(trainX_lightgbm, trainY_lightgbm)

# предсказания на валидационной выборке
predictions_lgbm = model_lgbm_best.predict(valX_lightgbm)

# RMSE на валидационной выборке
rmse_lgbm = np.sqrt(mean_squared_error(valY_lightgbm, predictions_lgbm))

print("RMSE for lgbm:", rmse_lgbm)

RMSE for lgbm: 1606.4859155622244
CPU times: user 3.54 s, sys: 1.41 s, total: 4.96 s
Wall time: 1.18 s


### Catboost

In [36]:
%%time

# категориальные признаки для модели CatBoost
cat_features = ["vehicle_type", "gearbox", "model", "fuel_type", "brand", "repaired"]

# CatBoostRegressor
model_catboost = CatBoostRegressor(loss_function='RMSE', random_seed=12345)

# гиперпараметры для подбора
params = {'depth': [6, 10],
          'learning_rate' : [0.01, 0.1],
          'iterations'    : [30, 100]}

# GridSearch
grid_search = GridSearchCV(estimator=model_catboost, param_grid = params, cv = 3, n_jobs=-1)
grid_search.fit(trainX, trainY, cat_features=cat_features, verbose=10)

# лучшие гиперпараметры
best_parameters = grid_search.best_params_
print("Best parameters: ", best_parameters)

0:	learn: 4294.0492920	total: 80ms	remaining: 7.92s
10:	learn: 2532.5740847	total: 417ms	remaining: 3.38s
20:	learn: 1990.8238290	total: 694ms	remaining: 2.61s
30:	learn: 1805.1324915	total: 950ms	remaining: 2.12s
40:	learn: 1742.2136860	total: 1.23s	remaining: 1.76s
50:	learn: 1705.6407656	total: 1.51s	remaining: 1.45s
60:	learn: 1682.8704123	total: 1.8s	remaining: 1.15s
70:	learn: 1659.7142180	total: 2.2s	remaining: 898ms
80:	learn: 1642.8920599	total: 2.47s	remaining: 579ms
90:	learn: 1628.3717722	total: 2.74s	remaining: 271ms
99:	learn: 1618.0144823	total: 3s	remaining: 0us
Best parameters:  {'depth': 10, 'iterations': 100, 'learning_rate': 0.1}
CPU times: user 13.5 s, sys: 1.44 s, total: 14.9 s
Wall time: 25.2 s


In [37]:
%%time
# Обучаем модель с использованием лучших гиперпараметров
model_catboost_best = CatBoostRegressor(depth=best_parameters['depth'],
                                        learning_rate=best_parameters['learning_rate'],
                                        iterations=best_parameters['iterations'],
                                        loss_function='RMSE', 
                                        random_seed=12345)

model_catboost_best.fit(trainX, trainY, cat_features=cat_features, verbose=10)

# предсказания на валидационной выборке
predictions_catboost = model_catboost_best.predict(valX)

# RMSE на валидационной выборке
rmse_catboost = np.sqrt(mean_squared_error(valY, predictions_catboost))

print("RMSE for CatBoost:", rmse_catboost)

0:	learn: 4294.0492920	total: 26ms	remaining: 2.58s
10:	learn: 2532.5740847	total: 310ms	remaining: 2.51s
20:	learn: 1990.8238290	total: 583ms	remaining: 2.19s
30:	learn: 1805.1324915	total: 833ms	remaining: 1.85s
40:	learn: 1742.2136860	total: 1.1s	remaining: 1.59s
50:	learn: 1705.6407656	total: 1.47s	remaining: 1.41s
60:	learn: 1682.8704123	total: 1.76s	remaining: 1.12s
70:	learn: 1659.7142180	total: 2.05s	remaining: 838ms
80:	learn: 1642.8920599	total: 2.34s	remaining: 549ms
90:	learn: 1628.3717722	total: 2.63s	remaining: 260ms
99:	learn: 1618.0144823	total: 2.89s	remaining: 0us
RMSE for CatBoost: 1683.551083279999
CPU times: user 12.7 s, sys: 1.09 s, total: 13.8 s
Wall time: 3.09 s


## Анализ моделей

На основании полученных результатов, мы можем проанализировать время обучения, время предсказания и качество моделей для всех трех моделей.

По качеству предсказаний (RMSE), обе модели градиентного бустинга - LightGBM и CatBoost - превосходят линейную регрессию. Однако, LightGBM показывает лучшие результаты по сравнению с CatBoost с точки зрения RMSE и скорости.

По времени обучения и времени предсказания, линейная регрессия является самой быстрой моделью. Однако, ее качество предсказаний оставляет желать лучшего. Среди моделей градиентного бустинга, LightGBM обучается и делает предсказания быстрее, чем CatBoost.

Таким образом, исходя из критериев заказчика (качество предсказания, время обучения и время предсказания), LightGBM является оптимальным выбором, так как она обеспечивает хорошее качество предсказаний при относительно быстром времени обучения и предсказания.

## Проверка на тестовой выборке

In [38]:
# Применяем лучшую модель (LightGBM) для предсказания цен автомобилей на тестовой выборке
test_preds_lightgbm = model_lgbm_best.predict(testX_lightgbm)

# Вычисляем RMSE на тестовой выборке
rmse_test_lightgbm = mean_squared_error(testY_lightgbm, test_preds_lightgbm, squared=False)
print("RMSE for LightGBM on test data: ", rmse_test_lightgbm)

RMSE for LightGBM on test data:  1617.9049222894987


## Общий вывод

В ходе этого проекта мы работали над задачей предсказания рыночной стоимости автомобилей для сервиса по продаже автомобилей с пробегом «Не бит, не крашен».

Мы обучили и оценили различные модели машинного обучения: линейную регрессию, LightGBM и CatBoost. Каждая из этих моделей была обучена и проверена с использованием тренировочных и тестовых данных.

По результатам обучения и тестирования, LightGBM показала наилучшую производительность. Эта модель не только обеспечивала наиболее точные предсказания (с наименьшим значением RMSE), но и обучалась и делала предсказания относительно быстро. RMSE на тестовых данных составил 1617.9, что значительно меньше установленного нами порога в 2500.

С точки зрения заказчика, которому важны качество предсказания, время обучения модели и время предсказания модели, LightGBM оказалась оптимальным выбором.

В заключение, на основе проведенного исследования, мы можем сказать, что использование модели градиентного бустинга LightGBM позволяет достичь хорошего качества предсказания стоимости автомобиля на вторичном рынке.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей